## Baracus

In [1]:
def get_single_subject(df_bar_subj, to_int = True):
    df_single_subject = pd.DataFrame(index=[x+1 for x in range(len(df_bar_subj))], columns = ['participant', 'aseg', 'area', 'thickness', 'stacked-anatomy'])
    i = 0
    # go through each participant
    for p in df_bar_subj.subj_path:
        tsv_file = [x for x in Path(p).glob('*.tsv')] # look at the predicted age tsv file that baracus outputs
        if tsv_file:
            df_temp = pd.read_csv(tsv_file[0], sep='\t')
            if to_int:
                df_single_subject.set_value(i, 'participant', int((df_temp.subject_id[0])[4:])) # get participant label
            else:
                df_single_subject.set_value(i, 'participant', (df_temp.subject_id[0])[4:]) # get participant label
            df_single_subject.set_value(i, 'aseg', df_temp.predicted_age[0]) # get aseg age prediction
            df_single_subject.set_value(i, 'area', df_temp.predicted_age[1]) # get area age prediction
            df_single_subject.set_value(i, 'thickness', df_temp.predicted_age[2]) # get thickness age prediction
            df_single_subject.set_value(i, 'stacked-anatomy', df_temp.predicted_age[3]) # get stacked age prediction
            i += 1
    
    return df_single_subject

## Machine Learning

### Function to Train Test Split

In [2]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
import pandas as pd

def train_test_pipeline(df_features, df_target, test_size=0.3, random_state=None, model='no',model_train = None, model_test=None, nonlinear=None):
    
    if model=='no':
        #split test and train data into equal parts
        X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size = test_size, random_state= random_state)
    
    else:
        df_features = df_features.sort_index()
        # get X_train
        X_train = []
        [X_train.append(df_features.iloc[x,:]) for x in model_train.index]
        X_train = pd.DataFrame(X_train)

        # get corresponding y_train
        y_train = pd.DataFrame([])
        y_train = y_train.assign(age_at_scan = [df_target.iloc[x] for x in model_train.index])
        y_train = y_train.set_index(model_train.index.values)
        
        # get X_test
        X_test = []
        [X_test.append(df_features.iloc[x,:]) for x in model_test.index]
        X_test = pd.DataFrame(X_test)

        # get corresponding y_test
        y_test = pd.DataFrame([])
        y_test = y_test.assign(age_at_scan = [df_target.iloc[x] for x in model_test.index])
        y_test = y_test.set_index(model_test.index.values)
        
    # imputer for missing values
    fill_missing = Imputer()
    var_thr = VarianceThreshold()
    normalize = StandardScaler() # standard nomalizer

    # create pipelist for sklearn transformations in order of specification
    pipeline_list = []
    pipeline_list = [('fill_missing', fill_missing),
                     ('var_thr', var_thr),
                     ('normalize', normalize)]
    
    # SVR regression model
    if not nonlinear:
        regression_model = SVR(kernel='linear', C=1.0, cache_size=1000)
    elif nonlinear == 'poly':
        regression_model = SVR(kernel=nonlinear, C=1.0, cache_size=1000)
    else:
        regression_model = SVR(kernel='rbf', C=1.0, cache_size=1000)
    
    pipeline_list.append(('regression_model', regression_model))
    
    # turn pipeline_list into sklearn Pipeline 
    pipe = Pipeline(pipeline_list)
    
    return X_train, X_test, y_train, y_test, pipe

/data/migineishvilin2/python/envs/100runs/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def pipe_fit(X_train, y_train):
    # create pipelist for sklearn transformations in order of specification
    # remove variance threshold because we're only using one feature
    # imputer for missing values
    fill_missing = Imputer()
    var_thr = VarianceThreshold()
    normalize = StandardScaler()
    regression_model = SVR(kernel='linear', C=1.0, cache_size=1000)
    
    pipeline_list = []
    pipeline_list = [('fill_missing', fill_missing),
                 ('var_thr', var_thr),
                 ('normalize', normalize),
                 ('regression_model', regression_model)]
    
    pipe = Pipeline(pipeline_list)
    pipe.fit(X_train, y_train)
    return pipe

## Plots

### Prediction vs. Age

In [3]:
# plots
import matplotlib.pyplot as plt
import numpy as np

# plots the train set performance on preduction v test set performance on prediction
def plot_pred(y_pred_train, y_target_train, y_pred_test, y_target_test, fig_tuple=(10, 8), context='notebook'):
    
    import seaborn as sns
    
    sns.set_context(context, font_scale=1.2)
    # plotting the Histogram
    plt.figure(1)
    plt.figure(figsize=fig_tuple)
    
    plt.plot(numpy.arange(0,80,0.1), numpy.arange(0,80,0.1), c='black', linewidth = 3)
    plt.scatter(y_pred_train, y_target_train, c='b', marker = 'v', label = 'train')
    plt.scatter(y_pred_test, y_target_test, color = 'r', marker = 'x', label='test')
    plt.legend(loc = 'upper left')
    plt.xlabel("predicted age")
    plt.ylabel("age")
    plt.xticks(np.arange(0,100,20))
    
    plt.show()

### Validation Curve

In [5]:
# Validation Curve
from sklearn.learning_curve import validation_curve
import pylab as plt

# Plots validation cure, i.e. C value versus Test Accuracy
def plot_validation_curve(pipe, X_train, y_train):
    param_range = np.logspace(-4, 2, num=18)
    # fixme n_jobs
    train_scores, test_scores = validation_curve(pipe, X_train, y_train, param_name="regression_model__C",
                                                     param_range=param_range)
    
    # plot
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.figure()
    plt.title("Validation Curve")
    plt.xlabel("C")
    plt.ylabel("Score")
    plt.ylim(0.0, 1.1)
    plt.semilogx(param_range, train_scores_mean, label="Training score", color="r")
    plt.fill_between(param_range, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std,
                         alpha=0.2,
                         color="r")
    plt.semilogx(param_range, test_scores_mean, label="Cross-validation score", color="g")
    plt.fill_between(param_range, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.2,
                         color="g")
    plt.legend(loc="best")
    plt.show()

/data/migineishvilin2/python/envs/100runs/lib/python3.6/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


### Learning Curve

In [6]:
# Plot Learning Curve as more Training set samples get added
def learning_curve_fct(X, y, out_name, estimator):
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.learning_curve import learning_curve
    import os, pickle
    from sklearn.svm import SVR
    from sklearn.linear_model import ElasticNet
    from sklearn.pipeline import Pipeline
    from sklearn.feature_selection import VarianceThreshold
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import Imputer


    def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                            n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
        plt.figure()
        plt.title(title)
        if ylim is not None:
            plt.ylim(*ylim)
            
        plt.xlabel("Training examples")
        plt.ylabel("Score")
        
        train_sizes, train_scores, test_scores = learning_curve(
            estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
        
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        
        plt.grid()
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1, color="g")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")

        plt.legend(loc="best")
        return plt
    
    title = 'Learning Curves: ' + out_name
    plot_learning_curve(estimator, title, X, y, cv=5, n_jobs=5, train_sizes=np.linspace(.5, 1.0, 5))
    plt.show()

### Baracus Data Plot

In [7]:
def bar_plot(df_bar_subj, start_at=2, end_at=7):
    import matplotlib.pyplot as plt
    import numpy as np

    # plot the predicted age v. real age of all modality predictions
    plt.plot(np.arange(0,120,0.1), np.arange(0,120,0.1), c='black', linewidth = 3)
    plt.scatter(df_bar_subj.iloc[:, start_at], df_bar_subj.iloc[:, end_at], c='b')
    plt.scatter(df_bar_subj.iloc[:, start_at+1], df_bar_subj.iloc[:, end_at], c='r', marker='x')
    plt.scatter(df_bar_subj.iloc[:, start_at+2], df_bar_subj.iloc[:, end_at], c='y', marker='s')
    plt.scatter(df_bar_subj.iloc[:, start_at+3], df_bar_subj.iloc[:, end_at], c='g', marker='v')
    plot_legend = plt.legend(loc = 'upper left')
    plot_legend.get_texts()[0].set_text("aseg")
    plot_legend.get_texts()[1].set_text("area")
    plot_legend.get_texts()[2].set_text("thickness")
    plot_legend.get_texts()[3].set_text("stacked")
    plt.xlabel("predicted age")
    plt.ylabel("age")
    plt.xticks(np.arange(0,300,20))
    plt.show()

## Freesurfer

In [73]:
import nibabel
import numpy

def vectorize_fs_surf(hemisphere_file, num_feat = None):
    image = nibabel.load(hemisphere_file) # load file
    sq_data = image.get_data().squeeze() # squeeze number of features
    
    if not num_feat:
        vectorized_data = sq_data[numpy.newaxis, ...] # save the squeezed data to vectorized_data
        assert vectorized_data.shape == (1, 2562) # assert that the new squeezed size
    else:
        sq_data = numpy.random.choice(sq_data, size= int((len(sq_data)*(0.75**num_feat))))
        vectorized_data = sq_data[numpy.newaxis, ...]

    return vectorized_data

In [121]:
def vectorize_fs_tab(aseg, num_feat = None):
    df_aseg = pd.read_csv(aseg, index_col=0, delimiter='\t') # read in aseg file
    
    if not num_feat:
        vectorized_data = df_aseg.values # save volume values in vectorized_data
        assert vectorized_data.shape == (1, 66) # assert that we have 66 volume measurements
    else:
        aseg_data = numpy.random.choice(df_aseg.values[0], size= int((len(df_aseg.values[0])*(0.75**num_feat))))
        vectorized_data = [aseg_data]
        
    return vectorized_data

In [77]:
def combine_surfaces(lh, rh, num_feat = None):
    lh_data = vectorize_fs_surf(lh, num_feat) # get squeezed vectors
    rh_data = vectorize_fs_surf(rh, num_feat)
    return np.concatenate((lh_data, rh_data), 1) # combine left and right hemispheres

In [78]:
# extract thickness, ages or area data from the features
def get_data(lh_thickness, rh_thickness, lh_area, rh_area, aseg, num_feat = None):
    X = {}
    X["thickness"] = combine_surfaces(lh_thickness, rh_thickness, num_feat) # vectorize cortical thickness
    X["area"] = combine_surfaces(lh_area, rh_area, num_feat) # vectorize cortical surface area
    X["aseg"] = vectorize_fs_tab(aseg, num_feat)
    
    return X

In [84]:
def get_features(subject, lh_thickness_file, rh_thickness_file, lh_area_file, rh_area_file, aseg_file, num_feat=None):
    X = get_data(lh_thickness_file, rh_thickness_file, lh_area_file, rh_area_file, aseg_file, num_feat = num_feat)
    return X

### Extract Data

In [80]:
import os

def subject_to_anal(subj_dirs, bids_bar):
    subjects_to_analyze = []
    # check baracus directory for extracted information from Freesurfer
    # if extracted information exists for that subject, continue
    # else, don't use that subject
    for s in subj_dirs:
        lh_thickness_file = os.path.join(bids_bar, s, "data", "lh.thickness.mgh")
        rh_thickness_file = os.path.join(bids_bar, s, "data", "rh.thickness.mgh")
        lh_area_file = os.path.join(bids_bar, s, "data", "lh.area.mgh")
        rh_area_file = os.path.join(bids_bar, s, "data", "rh.area.mgh")
        aseg_file = os.path.join(bids_bar, s, "data", "aseg")
    
        if os.path.isfile(lh_area_file) and os.path.isfile(rh_area_file) and os.path.isfile(lh_thickness_file) and os.path.isfile(rh_thickness_file) and aseg_file:
            subjects_to_analyze.append(s)
    
    return subjects_to_analyze

In [81]:
def extract_features(subjects_to_analyze, bids_bar, num_feat = None):
    features = []
    for s in subjects_to_analyze:
        # these are guaranteed to exists, since we removed subjects without these files in the previous for loop
        lh_thickness_file = os.path.join(bids_bar, s, "data", "lh.thickness.mgh")
        rh_thickness_file = os.path.join(bids_bar, s, "data", "rh.thickness.mgh")
        lh_area_file = os.path.join(bids_bar, s, "data", "lh.area.mgh")
        rh_area_file = os.path.join(bids_bar, s, "data", "rh.area.mgh")
        aseg_file = os.path.join(bids_bar, s, "data", "aseg")
    
        # get cortical thickness, cortical surface area and subcortical volumes
        temp_X = get_features(s, lh_thickness_file, rh_thickness_file, lh_area_file, rh_area_file, aseg_file, num_feat)
        temp_X["subj"] = s
        features.append(temp_X)
    
    return features

In [82]:
# get data
def get_source_data(source, features, target, on='MASKID', is_string=False):
    # Get Features
    assert source == 'thickness' or source == 'area' or source == 'aseg'
    arr_ = [x[source] for x in features]
    arr_ = [x[0] for x in arr_]
    narr_ = numpy.array(arr_) # covert into numpy array

    df_data_ = pd.DataFrame(narr_)

    if not is_string:
        df_data_[on] = pd.Series([int(x['subj'][4:]) for x in features], index=df_data_.index) # add subject number to df
    else:
        df_data_[on] = pd.Series([x['subj'][4:13] for x in features], index=df_data_.index)
    df_data_ = df_data_.merge(target, on=on) # add subject age to df
    return df_data_

## Stacking

In [83]:
def get_stacked_ages(y_train, y_predicted_train, y_predicted_cv, y_test, y_predicted_test, first=False):
    # stack predicted 
    
    # merge predicted ct for train 
    df_y_train = pd.DataFrame([])
    
    if not first:
        df_y_train = df_y_train.assign(age_at_scan = y_train.age_at_scan)
    else:
        df_y_train = df_y_train.assign(age_at_scan = y_train)
    
    df_y_train = df_y_train.set_index(y_train.index)
    df_y_train = df_y_train.assign(pred_age_test = y_predicted_train)
    df_y_train = df_y_train.assign(y_predicted_cv = y_predicted_cv)
    df_y_train = df_y_train.assign(split_group = "train")
    
    # merge predicted ct for test
    df_y_test = pd.DataFrame([])
    if not first:
        df_y_test = df_y_test.assign(age_at_scan = y_test.age_at_scan)
    else:
        df_y_test = df_y_test.assign(age_at_scan = y_test)
    df_y_test = df_y_test.set_index(y_test.index)
    df_y_test = df_y_test.assign(pred_age_test = y_predicted_test)
    df_y_test = df_y_test.assign(split_group = "test")
    
    # concat ct test and ct train together
    df_y = pd.concat([df_y_train, df_y_test])
    
    # sort by index to make sure we have the age predictions are in the right order,
    # since they have been randomly shuffled
    df_y = df_y.sort_index()
    
    return df_y

In [17]:
import os
import pickle
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cross_validation import cross_val_predict, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error

def tune_and_train_rf(X_train, y_train, target, strat_k_fold=None):
    '''
    Uses oob estimates to find optimal max_depth between None + 0...20
    Refits with best max_depth
    '''
    oob_r2 = []
    cv_list = [x for x in range(1, 20)] # different RF Depths
    for md in cv_list:
        # generate random forests and determine best depth
        rf = RandomForestRegressor(n_estimators=100, max_depth=md, oob_score=True, random_state=0, n_jobs=-1)
        rf.fit(X_train, y_train)
        oob_r2.append(rf.oob_score_)

    best_max_depth = cv_list[np.argmax(oob_r2)]
    print("best max_depth: %s" % best_max_depth)

    # CV
    # get random forest model
    rf = RandomForestRegressor(n_estimators=100, max_depth=best_max_depth, oob_score=True, random_state=0, n_jobs=-1)
    
    cv_results = None
    if strat_k_fold:
        y_predicted_cv = cross_val_predict(rf, X_train, y_train, cv=strat_k_fold, n_jobs=-1)
        cv_r2 = []
        cv_mae = []
        for k_train, k_test in strat_k_fold:
            cv_r2.append(r2_score(y_train[k_test], y_predicted_cv[k_test]))
            cv_mae.append(mean_absolute_error(y_train[k_test], y_predicted_cv[k_test]))
        cv_results = {'y_predicted_cv': y_predicted_cv,
                      'cv_r2': cv_r2,
                      'cv_mae': cv_mae,
                      'oob_r2': oob_r2}

    # refit
    rf.fit(X_train, y_train)
    return rf, cv_results

In [18]:
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.cross_validation import cross_val_predict, StratifiedKFold

def stacking(source_dict, source_selection_dict, target, run_fitting=True, show=True):
    df_in = {}

    # create df_in for ct, ca and sv and add a source column to identify ct, ca or sv
    for s, f in source_dict.items():
        df_in[s] = f
        df_in[s]['source'] = s
        
    # go though fs key
    for stacking_crit in source_selection_dict.keys():
        file_pref = target + '__' + stacking_crit + '__'
        scores_test = pd.DataFrame([])
        df_all = pd.DataFrame([]) # empty df
    
        # concatenate ct, ca, sv vertically
        for s in source_selection_dict[stacking_crit]:
            df_all = pd.concat((df_all, df_in[s]))
    
        # get one single source example to get age...
        # gets ct because it's the last item in the key
        df_single_source = df_in[s]
    
        # add columns in the case of test-only data
        if 'split_group' not in df_all:
            df_all['split_group'] = 'test'
    
        # add 'select', 'y_pred_cv', 'sample_weights', 'train_group_2samp' and 'study if those fields not already provided
        for a in ['select', 'y_predicted_cv','sample_weights', 'train_group_2samp', 'study']:
            if a not in df_all:
                df_all[a] = np.nan
    
        # create a df that only takes certain columns of df_all
        df = df_all[['source', 'age_at_scan', 'split_group', 'select', 'y_predicted_cv', 'pred_age_test', 'sample_weights', 'train_group_2samp', 'study']]
        
        
        # separate df into test subjects
        test_ind = df['split_group'] == 'test'
        df_test = df[test_ind].copy()
    
        # run_fitting on train data 
        if run_fitting:  # fit rf
            print ('Fitting stacking model')
        
            # split df into train subjects
            train_ind = ((df['split_group'] == 'train') | (df['train_group_2samp'] == True))
            df_train = df[train_ind].copy() 
            
            # pivot table so that df_train displays index as rows, aseg, ca and ct as columns and y_pred_cv for corresponding values
            dd_train = df_train.pivot_table(values='y_predicted_cv', columns='source', index=df_train.index)
#             dd_train = pd.DataFrame([])
#             dd_train = dd_train.assign(aseg = [df_train.y_predicted_cv[x] if df_train.source[x] == 'aseg' else None for x in df_train.index])
            
            # get ['aseg', 'ca', 'ct']
            single_sources = dd_train.columns.values
            # get mean y_pred across the three sources
            dd_train['mean_pred'] = dd_train.mean(1)
            # add actual age in
            dd_train = dd_train.join(df_single_source[['age_at_scan']], how='left')
        
            # split into training features and target
            X_train, y_train = dd_train[single_sources], dd_train['age_at_scan']
        
            # Call Tuning and Training Random Forest Function
            rf, cv_results = tune_and_train_rf(X_train, y_train, target)
        
            # Plot Feature Importance
            if show:
                fi = pd.DataFrame(rf.feature_importances_, columns=['feature_importances'], index=single_sources)
                plt.figure()
                sns.barplot(fi.feature_importances, fi.index.values)
                plt.xlim([0, 1])
                plt.show()
        
            # Get Training Error
            y_predicted_train_stack = rf.predict(X_train)
            r2_train = r2_score(y_train, y_predicted_train_stack)
            mae_train = mean_absolute_error(y_train, y_predicted_train_stack)
            
            dd_train['pred_age_train'] = y_predicted_train_stack
            
            if show:
                plt.figure()
                f = sns.jointplot('age_at_scan', 'pred_age_train', data=dd_train, xlim=(10, 90), ylim=(10, 90))
                ax = sns.plt.gca()
                plt.show()
        else:
            rf_file = rf_file_template.format(stacking_crit=stacking_crit)
            rf = pickle.load(open(rf_file))
            dd_train = pd.DataFrame([])
        
        # Prediction on Test Set
        
        # Pivot Test Subject so that columns = aseg, ca, ct and rows are the index of test subjects and values are predicted ages
        dd_test = df_test.pivot_table(values='pred_age_test', columns='source', index=df_test.index)
    
        # get ['aseg', 'ca', 'ct']
        single_sources = dd_test.columns.values
    
        dd_test['mean_pred'] = dd_test.mean(1) # calculate mean predicted age
        dd_test = dd_test.join(df_single_source[['age_at_scan']], how='left') # add real age at the end of the df
    
        # separate into test features and test target
        X_test, y_test = dd_test[single_sources], dd_test['age_at_scan']
        dd_test['pred_age_test'] = rf.predict(X_test)
    
        for m in source_selection_dict[stacking_crit] + ['mean_pred', 'pred_age_test']:
            scores_test.ix[m, 'r2'] = r2_score(dd_test['age_at_scan'], dd_test[m])
            scores_test.ix[m, 'rpear'] = np.corrcoef(dd_test['age_at_scan'], dd_test[m])[0, 1]
            scores_test.ix[m, 'rpear2'] = np.corrcoef(dd_test['age_at_scan'], dd_test[m])[0, 1] ** 2
            scores_test.ix[m, 'mae'] = mean_absolute_error(dd_test['age_at_scan'], dd_test[m])
            scores_test.ix[m, 'medae'] = median_absolute_error(dd_test['age_at_scan'], dd_test[m])
            
            if show:
                plt.figure()
                plt.scatter(dd_test['age_at_scan'], dd_test[m])
                plt.plot([10, 90], [10, 90])
                plt.xlim([10, 90])
                plt.ylim([10, 90])
                plt.title('predictions TEST: %s (%s)\n%.3f' % (m, stacking_crit, scores_test.ix[m, 'r2']))
                plt.gca().set_aspect('equal', adjustable='box')
                plt.show()
        
        return scores_test, dd_train, dd_test, rf

## Anatomical Data

In [19]:
def get_anat_features(subjects_to_analyze, bids_bar):
    white_matter = []
    grey_matter = []
    csf = []
    intra_cran_vol = []

    for s in subjects_to_analyze:
        # calculate white matter
        df_tempsub = pd.DataFrame.from_csv(bids_bar.joinpath(s + "/data/aseg").as_posix(), sep='\t')
        white_matter_ms = df_tempsub.iloc[0]['Left-Cerebellum-White-Matter'] + df_tempsub.iloc[0]['Right-Cerebellum-White-Matter'] + df_tempsub.iloc[0]['CorticalWhiteMatterVol']
        white_matter.append(white_matter_ms)
    
        # calculate grey matter
        grey_matter_ms = df_tempsub.iloc[0]['SubCortGrayVol']
        grey_matter.append(grey_matter_ms)
    
        # calculate csf
        csf_ms = df_tempsub.iloc[0]['BrainSegVol'] - df_tempsub.iloc[0]['BrainSegVolNotVent']
        #csf_ms = df_tempsub.iloc[0]['CSF']
        csf.append(csf_ms)
    
        # calculate intracranial volume
        intra_cran_vol_ms = df_tempsub.iloc[0]['EstimatedTotalIntraCranialVol']
        intra_cran_vol.append(intra_cran_vol_ms)
        
    return white_matter, grey_matter, csf, intra_cran_vol

In [20]:
def calc_fraction(matter, total):
    for indx in range(len(matter)):
        matter[indx] = matter[indx]/total[indx]
        
    return matter

In [21]:
import shutil
import os
# Copy Files Over, then Run Baracus 
def copy_hcp_files(hcp_fs_surf_subj, hcp_fs_dir, hcp_fs_stats_subj):
    for x in hcp_fs_surf_subj:
        # get the right surf files
        for f in ['lh.area', 'lh.sphere.reg', 'lh.thickness','rh.area', 'rh.sphere.reg', 'rh.thickness']:
            out_dir = hcp_fs_dir.joinpath('sub-'+x[len(x)-11:len(x)-5]+'/surf/').as_posix() # out dir in NNDSP
            # check if surf file exists, or if it has already been copied over
            if (Path(x).joinpath(f)).exists() and not Path(out_dir).joinpath(f).exists():
                print("Copying: ", Path(x).joinpath(f), " to", out_dir)
                if not os.path.isdir(out_dir):
                    os.makedirs(out_dir)
                shutil.copy(Path(x).joinpath(f).as_posix() , out_dir)

    for x in hcp_fs_stats_subj:
        # get aseg files
        for f in ['aseg.stats']:
            out_dir = hcp_fs_dir.joinpath('sub-'+x[len(x)-12:len(x)-6]+'/stats/').as_posix() # out dir in NNDSP        
            #check if aseg file exists, or if it has already been copied over
            if (Path(x).joinpath(f)).exists() and not Path(out_dir).joinpath(f).exists():
                print("Copying: ", Path(x).joinpath(f), " to", out_dir)
                if not os.path.isdir(out_dir):
                    os.makedirs(out_dir)
                shutil.copy(Path(x).joinpath(f).as_posix() , out_dir)

## Util

In [22]:
from sklearn.metrics import mean_absolute_error
def print_mae(y_target, y_pred, model="", run_type=''):
    print("Mean Absolute Error", model, run_type, mean_absolute_error(y_target, y_pred))